In [33]:
import os
import json
import sys 
from sys import exit 
import numpy as np

In [34]:
import time
import urllib.request
import ssl
from dotenv import load_dotenv
from unicodedata import name
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from urllib.parse import urlparse, parse_qs

In [35]:
load_dotenv()

True

In [36]:
twitter_user = os.getenv("twitter_user")
twitter_password = os.getenv("twitter_password")

service = Service('E:/Users/1167486/Local/Drivers_web/chromedriver-win64/chromedriver')
options = webdriver.ChromeOptions()
options.binary_location = r'E:/Users/1167486/Local/Drivers_web/chrome-win64/chrome.exe'
options.add_argument('--disable-blink-features=AutomationControlled')
driver = webdriver.Chrome(service=service, options=options)


In [37]:
driver.maximize_window()
driver.get('https://x.com/i/flow/login')
time.sleep(7)

In [38]:
username = driver.find_element("css selector", "input[autocomplete='username']")


In [39]:
username.clear()


In [40]:
username.send_keys(twitter_user)
username.send_keys(Keys.ENTER)
time.sleep(5)

In [41]:
password = driver.find_element("css selector", "input[autocomplete='current-password']")
password.clear()
password.send_keys(twitter_password)
password.send_keys(Keys.ENTER)



In [42]:
#tweet_test = 'https://x.com/anabelhoficial/status/1906888988192887030'
#tweet_test = 'https://x.com/Juan_OrtizMX/status/1907289700101935494'
tweet_test = 'https://x.com/RicardoBSalinas/status/1914398783624126967'

In [ ]:
#driver.get(tweet_test)

In [43]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support   import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd
import re
from bs4 import BeautifulSoup

In [44]:
## Ejecutar esto siempre al testear
link = tweet_test
driver.get(link)

In [ ]:
#'<button role="button" class="css-175oi2r r-1777fci r-1pl7oy7 r-13qz1uu r-1loqt21 r-o7ynqc r-6416eg r-1ny4l3l" type="button"><div class="css-175oi2r r-16y2uox r-1wbh5a2 r-1777fci"><div dir="ltr" class="css-146c3p1 r-bcqeeo r-qvutc0 r-37j5jr r-q4m81j r-a023e6 r-rjixqe r-16dba41" style="color: rgb(29, 155, 240);"><span class="css-1jxf684 r-bcqeeo r-1ttztb7 r-qvutc0 r-poiln3">Show probable spam</span></div></div></button>'
#'#react-root > div > div > div.css-175oi2r.r-1f2l425.r-13qz1uu.r-417010.r-18u37iz > main > div > div > div > div > div > section > div > div > div:nth-child(18) > div.css-175oi2r.r-1adg3ll.r-1ny4l3l > div > button'

In [45]:
boton_xpath = '//*[@id="react-root"]/div/div/div[2]/main/div/div/div/div/div/section/div/div/div[12]/div[1]/div/button/div/div/span'
boton_xpath2 = '/html/body/div[1]/div/div/div[2]/main/div/div/div/div/div/section/div/div/div[22]/div[1]/div/button/div'
#'//*[@id="react-root"]/div/div/div[2]/main/div/div/div/div/div/section/div/div/div[20]/div[1]/div/button/div'
#'//*[@id="react-root"]/div/div/div[2]/main/div/div/div/div/div/section/div/div/div[20]/div[1]/div/button/div/div'
'/html/body/div[1]/div/div/div[2]/main/div/div/div/div/div/section/div/div/div[20]/div[1]/div/button/div/div/span'
boton_cssselector = '#react-root > div > div > div.css-175oi2r.r-1f2l425.r-13qz1uu.r-417010.r-18u37iz > main > div > div > div > div > div > section > div > div > div:nth-child(20) > div.css-175oi2r.r-1adg3ll.r-1ny4l3l > div > button > div > div > span'
boton_cssselector = '#react-root > div > div > div.css-175oi2r.r-1f2l425.r-13qz1uu.r-417010.r-18u37iz > main > div > div > div > div > div > section > div > div > div:nth-child(20) > div.css-175oi2r.r-1adg3ll.r-1ny4l3l > div > button > div'


In [ ]:
#By.CSS_SELECTOR

In [14]:
'''
for i in range(12):
    print(f'Intento número: {i + 1}')
    try:
        #boton_mostrar_mas = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, boton_xpath)))
        #boton_mostrar_mas = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, boton_xpath2)))    
        boton_mostrar_mas = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, boton_cssselector)))    
        ### Una vez que se encuentra el botón, se hace clic en él
        boton_mostrar_mas.click()
        print('Encontró el botón "Probable Spam"')
        time.sleep(3)  # Espera a que se carguen más tweets después de hacer clic
    except:
        print('Scroll Normal')
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
        time.sleep(3)
'''

'\nfor i in range(12):\n    print(f\'Intento número: {i + 1}\')\n    try:\n        #boton_mostrar_mas = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, boton_xpath)))\n        #boton_mostrar_mas = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, boton_xpath2)))    \n        boton_mostrar_mas = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, boton_cssselector)))    \n        ### Una vez que se encuentra el botón, se hace clic en él\n        boton_mostrar_mas.click()\n        print(\'Encontró el botón "Probable Spam"\')\n        time.sleep(3)  # Espera a que se carguen más tweets después de hacer clic\n    except:\n        print(\'Scroll Normal\')\n        driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")\n        time.sleep(3)\n'

In [ ]:
##//*[@id="react-root"]/div/div/div[2]/main/div/div/div/div/div/section/div/div/div[12]/div[1]/div/button/div/div/span

In [46]:
boton_xpath = '//*[@id="react-root"]/div/div/div[2]/main/div/div/div/div/div/section/div/div/div[12]/div[1]/div/button/div/div/span'
boton_cssselector = '#react-root > div > div > div.css-175oi2r.r-1f2l425.r-13qz1uu.r-417010.r-18u37iz > main > div > div > div > div > div > section > div > div > div:nth-child(20) > div.css-175oi2r.r-1adg3ll.r-1ny4l3l > div > button > div'
#'#react-root > div > div > div.css-175oi2r.r-1f2l425.r-13qz1uu.r-417010.r-18u37iz > main > div > div > div > div > div > section > div > div > div:nth-child(18) > div.css-175oi2r.r-1adg3ll.r-1ny4l3l > div > button > div > div > span'
### Tweets Counter 

try:
    link = tweet_test
    driver.get(link)
    all_tweets_data = []  # Lista para almacenar los datos de cada tweet
    max_scroll_attempts = 20  # Número máximo de intentos de scroll/clic
    scroll_attempt = 0
    previous_tweet_count = 0
    no_new_tweets_threshold = 20  # Si no se encuentran nuevos tweets después de N intentos, detener
    no_new_tweets_counter=0
    while scroll_attempt < max_scroll_attempts:
        scroll_attempt += 1
        print(f'Intento de carga número: {scroll_attempt}')

        try:
            # Intenta encontrar y hacer clic en el botón
            boton_mostrar_mas = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, boton_cssselector))
            )
            print('Encontró el botón "Probable Spam"')  
            boton_mostrar_mas.click()
            time.sleep(4)
        except:
            # Si el botón no se encuentra o no es clickable, intenta hacer scroll
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
            print('Haciendo scroll normal.')  

            time.sleep(4)

        html_content = driver.page_source
        soup = BeautifulSoup(html_content, 'html.parser')
        #tweet_text_elements = soup.find_all('div', {'data-testid': 'tweetText'})
        tweet_text_elements = soup.find_all('div', {'data-testid': 'tweetText'})
        #tweet_text_elements = soup.find_all('div', {'data-testid': 'tweet'})
         
        current_tweet_count = len(tweet_text_elements)

        if current_tweet_count > previous_tweet_count:
            print(f"Se encontraron {current_tweet_count - previous_tweet_count} nuevos tweets.")
            for tweet_element in tweet_text_elements[previous_tweet_count:]:
                tweet_text = tweet_element.text.strip().replace("\n", "")
                all_tweets_data.append({'Texto del Tweet': tweet_text})
            previous_tweet_count = current_tweet_count
            no_new_tweets_counter = 0  
        else:
            no_new_tweets_counter += 1
            print(f"No se encontraron nuevos tweets en este intento ({no_new_tweets_counter}/{no_new_tweets_threshold}).")
            if no_new_tweets_counter >= no_new_tweets_threshold:
                print("Parece que se ha llegado al final de los tweets cargados.")
                break

    # Crea el DataFrame de pandas a partir de la lista de diccionarios
    df_tweets = pd.DataFrame(all_tweets_data)

    # Imprime información sobre el DataFrame resultante
    print(f"\nSe descargaron {len(df_tweets)} tweets.")
    # print(df_tweets) # Descomentar para ver el DataFrame completo

except Exception as e:
    print(f"Ocurrió un error: {e}")
finally:
    driver.quit()

Intento de carga número: 1
Haciendo scroll normal.
Se encontraron 13 nuevos tweets.
Intento de carga número: 2
Haciendo scroll normal.
Se encontraron 5 nuevos tweets.
Intento de carga número: 3
Haciendo scroll normal.
Se encontraron 1 nuevos tweets.
Intento de carga número: 4
Haciendo scroll normal.
Se encontraron 1 nuevos tweets.
Intento de carga número: 5
Haciendo scroll normal.
No se encontraron nuevos tweets en este intento (1/20).
Intento de carga número: 6
Haciendo scroll normal.
Se encontraron 5 nuevos tweets.
Intento de carga número: 7
Haciendo scroll normal.
No se encontraron nuevos tweets en este intento (1/20).
Intento de carga número: 8
Haciendo scroll normal.
No se encontraron nuevos tweets en este intento (2/20).
Intento de carga número: 9
Haciendo scroll normal.
No se encontraron nuevos tweets en este intento (3/20).
Intento de carga número: 10
Haciendo scroll normal.
No se encontraron nuevos tweets en este intento (4/20).
Intento de carga número: 11
Haciendo scroll norm

In [ ]:
'''
try:
    link = tweet_test
    driver.get(link)
    all_tweets_data = []  # Lista para almacenar los datos de cada tweet
    num_scrolls = 90  # Número máximo de scrolls
    number = 0
    for _ in range(num_scrolls):
        number += 1
        print(f'Número de scrolls: {number}')
        # Espera a que al menos un nuevo elemento 'tweetText' esté presente
        WebDriverWait(driver, 45).until(
            EC.presence_of_element_located((By.XPATH, '//div[@data-testid="tweetText"]'))
        )

        # Intenta encontrar y hacer clic en el botón
        try:
            boton_mostrar_mas = WebDriverWait(driver, 10).until(
                #EC.element_to_be_clickable((By.XPATH, boton_xpath))
                EC.element_to_be_clickable((By.CSS_SELECTOR, boton_cssselector))                
            )
            boton_mostrar_mas.click()
            time.sleep(3)  # Espera a que se carguen más tweets después de hacer clic
        except:
            # Si el botón no se encuentra o no es clickable, continúa con el scroll
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
            time.sleep(3)

        html_content = driver.page_source
        soup = BeautifulSoup(html_content, 'html.parser')
        tweet_text_elements = soup.find_all('div', {'data-testid': 'tweetText'})

        for tweet_element in tweet_text_elements:
            tweet_text = tweet_element.text.strip().replace("\n", "")
            all_tweets_data.append({'Texto del Tweet': tweet_text})

    # Crea el DataFrame de pandas a partir de la lista de diccionarios
    df_tweets = pd.DataFrame(all_tweets_data)

    # Imprime el DataFrame (opcional)
    #print(df_tweets)

except Exception as e:
    print(f"Ocurrió un error: {e}")
finally:
    driver.quit()
'''

Número de scrolls: 1
Número de scrolls: 2
Número de scrolls: 3
Número de scrolls: 4
Número de scrolls: 5
Número de scrolls: 6
Número de scrolls: 7
Número de scrolls: 8
Número de scrolls: 9
Número de scrolls: 10
Número de scrolls: 11
Número de scrolls: 12
Número de scrolls: 13
Número de scrolls: 14
Número de scrolls: 15
Número de scrolls: 16
Número de scrolls: 17
Número de scrolls: 18
Número de scrolls: 19
Número de scrolls: 20
Número de scrolls: 21
Número de scrolls: 22
Número de scrolls: 23
Número de scrolls: 24
Número de scrolls: 25
Número de scrolls: 26
Número de scrolls: 27
Número de scrolls: 28
Número de scrolls: 29
Número de scrolls: 30
Número de scrolls: 31
Número de scrolls: 32
Número de scrolls: 33
Número de scrolls: 34
Número de scrolls: 35
Número de scrolls: 36
Número de scrolls: 37
Número de scrolls: 38
Número de scrolls: 39
Número de scrolls: 40
Número de scrolls: 41
Número de scrolls: 42
Número de scrolls: 43
Número de scrolls: 44
Número de scrolls: 45
Número de scrolls: 

In [ ]:
'''
try:
    link = tweet_test
    driver.get(link)
    all_tweets_data = []  # Lista para almacenar los datos de cada tweet

    for _ in range(50):  # Ajusta el número de scrolls según sea necesario
        # Espera a que al menos un nuevo elemento 'tweetText' esté presente
        WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, '//div[@data-testid="tweetText"]'))
        )
        html_content = driver.page_source
        soup = BeautifulSoup(html_content, 'html.parser')
        tweet_text_elements = soup.find_all('div', {'data-testid': 'tweetText'})

        for tweet_element in tweet_text_elements:
            tweet_text = tweet_element.text.strip().replace("\n", "")
            all_tweets_data.append({'Texto del Tweet': tweet_text})

        driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
        time.sleep(3) # Un pequeño tiempo de espera después del scroll

    # Crea el DataFrame de pandas a partir de la lista de diccionarios
    df_tweets = pd.DataFrame(all_tweets_data)

    # Imprime el DataFrame (opcional)
    #print(df_tweets)

except Exception as e:
    print(f"Ocurrió un error: {e}")
finally:
    driver.quit()
'''

In [122]:
df_tweets.shape

(27, 1)

In [123]:
df_tweets.tail()

,Texto del Tweet
22,"Lo que usted Tio me quiera regalar, para pagar..."
23,"Neither gold nor Bitcoin is inherently ""better..."
24,Oro
25,Bitcoin y tu?
26,No pongas tu mundo de cabeza. Con la app de in...


In [124]:
df_tweets.to_csv('tweets2.csv', index=False, encoding='utf-8-sig')

In [38]:
'''
last_height = driver.execute_script("return document.body.scrollHeight")
while True:
    # Scroll hasta el fin 
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")

    # Espera a que la página se cargue 
    time.sleep(4)


    # Recalcula la altura de la página
    new_height = driver.execute_script("return document.body.scrollHeight")

    # Si ya no aumenta la altura, se termina el bucle
    if new_height == last_height:
        break
    last_height = new_height
'''

'\nlast_height = driver.execute_script("return document.body.scrollHeight")\nwhile True:\n    # Scroll hasta el fin \n    driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")\n\n    # Espera a que la página se cargue \n    time.sleep(4)\n\n\n    # Recalcula la altura de la página\n    new_height = driver.execute_script("return document.body.scrollHeight")\n\n    # Si ya no aumenta la altura, se termina el bucle\n    if new_height == last_height:\n        break\n    last_height = new_height\n'

In [50]:
from bs4 import BeautifulSoup


In [ ]:
'''
try:
    link = tweet_test
    driver.get(link)
    tweet_text_elements = []
    # Scroll 5 times to load more tweets
    for _ in range(45):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
        time.sleep(5) 
        html_content = driver.page_source
        soup = BeautifulSoup(html_content, 'html.parser')
        # Find all elements with 'tweetText'
        tweet_text_elements.extend(soup.find_all('div', {'data-testid': 'tweetText'}))
    # first 10 tweet
    for i, tweet_text_element in enumerate(tweet_text_elements):
        tweet_text = tweet_text_element.text.strip().replace("\n", "")
        print(f"Tweet {i+1}: {tweet_text}")
finally:
    driver.quit()
'''

Tweet 1: ¿Que prefieren #Bitcoin u oro?
Tweet 2: ¿Que prefieren #Bitcoin u oro?
Tweet 3: ¿Que prefieren #Bitcoin u oro?
Tweet 4: ¿Que prefieren #Bitcoin u oro?
Tweet 5: ¿Que prefieren #Bitcoin u oro?
Tweet 6: ¿Que prefieren #Bitcoin u oro?
Tweet 7: ¿Que prefieren #Bitcoin u oro?
Tweet 8: ¿Que prefieren #Bitcoin u oro?
Tweet 9: ¿Que prefieren #Bitcoin u oro?
Tweet 10: ¿Que prefieren #Bitcoin u oro?


In [52]:
len(tweet_text_elements)

305

In [56]:
import pandas as pd 
df = pd.DataFrame({'col':tweet_text_elements})
print(df)

df.to_csv('tweets.csv', index=False, encoding='utf-8-sig')

                                                   col
0         [[¿Que prefieren ], [[#Bitcoin]], [ u oro?]]
1         [[¿Que prefieren ], [[#Bitcoin]], [ u oro?]]
2         [[¿Que prefieren ], [[#Bitcoin]], [ u oro?]]
3         [[¿Que prefieren ], [[#Bitcoin]], [ u oro?]]
4         [[¿Que prefieren ], [[#Bitcoin]], [ u oro?]]
..                                                 ...
300  [["El capitalismo desenfrenado"(NEOLIBERALISMO...
301                                            [[Oro]]
302                                            [[ORO]]
303                                            [[Oro]]
304                                            [[Oro]]

[305 rows x 1 columns]


In [59]:
import os 
os.currentdir = os.getcwd()
print(os.currentdir)

e:\Users\1167486\Local\scripts\Social_media_comments\test


In [53]:
for i, tweet_text_element in enumerate(tweet_text_elements):
    print(tweet_text_element.text.strip().replace("\n", ""))

¿Que prefieren #Bitcoin u oro?
¿Que prefieren #Bitcoin u oro?
¿Que prefieren #Bitcoin u oro?
¿Que prefieren #Bitcoin u oro?
¿Que prefieren #Bitcoin u oro?
¿Que prefieren #Bitcoin u oro?
¿Que prefieren #Bitcoin u oro?
¿Que prefieren #Bitcoin u oro?
¿Que prefieren #Bitcoin u oro?
¿Que prefieren #Bitcoin u oro?
¿Que prefieren #Bitcoin u oro?
¿Que prefieren #Bitcoin u oro?
¿Que prefieren #Bitcoin u oro?
¿Que prefieren #Bitcoin u oro?
¿Que prefieren #Bitcoin u oro?
¿Que prefieren #Bitcoin u oro?
¿Que prefieren #Bitcoin u oro?
¿Que prefieren #Bitcoin u oro?
¿Que prefieren #Bitcoin u oro?
¿Que prefieren #Bitcoin u oro?
¿Que prefieren #Bitcoin u oro?
¿Que prefieren #Bitcoin u oro?
¿Que prefieren #Bitcoin u oro?
¿Que prefieren #Bitcoin u oro?
¿Que prefieren #Bitcoin u oro?
¿Que prefieren #Bitcoin u oro?
¿Que prefieren #Bitcoin u oro?
¿Que prefieren #Bitcoin u oro?
¿Que prefieren #Bitcoin u oro?
¿Que prefieren #Bitcoin u oro?
¿Que prefieren #Bitcoin u oro?
¿Que prefieren #Bitcoin u oro?
¿Que pre

In [21]:
from bs4 import BeautifulSoup
html_content = driver.page_source
soup = BeautifulSoup(html_content, 'html.parser')
x = soup.find_all('div', {'data-testid': 'tweetText'})



In [22]:
for i, tweet_text_element in enumerate(x):
    tweet_text = tweet_text_element.text.strip().replace("\n", "")
    print(f"Tweet {i+1}: {tweet_text}")

Tweet 1: Un millón de veces mejor el oro.
Tweet 2: Bien simple sólo paga y de ahí a chingar a su madre
Tweet 3: Bitcoin
Tweet 4: Oro
Tweet 5: "El capitalismo desenfrenado"(NEOLIBERALISMO)es un capitalismo delincuencial donde ya no hay ética ni principios humanos, lo mismo son secuestradores, corruptos, o evasores de impuestos, Eje: hermana de  @XochitlGalvez o @RicardoBSalinas@CarlosLoret @aristeguicnn @lopezdoriga
Tweet 6: ORO
Tweet 7: Oro
Tweet 8: Oro


In [ ]:
'''
try:
    link = 'https://twitter.com/elonmusk'
    driver.get(link)

    tweet_text_elements = []

    # Scroll 5 times to load more tweets
    for _ in range(5):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
        time.sleep(2) 

        html_content = driver.page_source
        soup = BeautifulSoup(html_content, 'html.parser')

        # Find all elements with 'tweetText'
        tweet_text_elements.extend(soup.find_all('div', {'data-testid': 'tweetText'}))

    # first 10 tweet
    for i, tweet_text_element in enumerate(tweet_text_elements[:10]):
        tweet_text = tweet_text_element.text.strip().replace("\n", "")
        print(f"Tweet {i+1}: {tweet_text}")

finally:
    driver. Quit()
'''

In [ ]:
comments = driver.find_elements("css selector", "div[data-testid='tweetText']")
comments[0].text
#driver.find_element(By.ID, 'tweetText')
len(comments)

8

In [17]:
tweets = driver.find_elements("css selector", "article[data-testid='tweet']")
tweets[0].text
len(tweets)


7

In [60]:
len(tweets)


3

In [48]:
len(tweets)

17

In [37]:
#names = driver.find_elements('css selector', "span[class='css-1jxf684 r-bcqeeo r-1ttztb7 r-qvutc0 r-poiln3']")
names = driver.find_elements("css selector", "div[data-testid='User-Name']")

#names = driver.find_elements(By.XPATH, 
#                             '//*[@id="id__8hzjlqcfwha"]/div[1]/div/a/div/div[1]/span/span')

In [38]:
for x in names:
    print(x.text)


Anabel Hernández
@anabelhoficial
xrp.gerry.xah
@gerryxrp
·
Mar 31
Grok
@grok
·
Mar 31
Rossana Villares
@RossanaVillares
·
22h
Tere
@Teehreh
·
16h
Armando Galindo Reyes
@armandgal65
·
2h
Gilberto Astorga F
@vzlaastor2025
·
13h
Rena Cornejo
@cornejo_rena
·
23h
Juan Carlos Cortés
@jccortes
·
Apr 1
Kubala
@cubalamx
·
12h
Victor manuel Vazquez guerrero
@Victorm06387347
·
12h
Astrophisic
@culiacan929
·
Mar 31
Nadia
@politicanadia
·
18h
Luz Maria Saltijeral
@Luzmasaltijeral
·
Apr 1


In [25]:
#comments = driver.find_elements("css selector", "div[id='tweetText']")


In [26]:
for comment in comments:
    print(comment.text)




Siempre estaré agradecida con cada uno de mis lectores, por aquellos que decidieron conocer la verdad de los hechos ante las situaciones que afectan al país. Gracias por apreciar mi trabajo en "La historia secreta: AMLO y el Cártel de Sinaloa".
Hey 
@grok
 los libros de 
@anabelhoficial
 son ciencia ficción?
No, los libros de Anabel Hernández no son ciencia ficción. Son obras de periodismo investigativo basadas en hechos reales, como "La Historia Secreta: AMLO y el Cártel de Sinaloa", que explora corrupción y narcotráfico en México con evidencia documental, no elementos
Anabel estás correspondida !
La periodista  Anavel si pasara a la historia  de México.
Excelente libro
#PobreBurra3D
Señora,
¿No le gustaría conocer a Jaime Maussan.
Pienso que tendrían mucho de que platicar.
                                    
Parece que se acabó el contrato con la editorial... ¿Es despedía? ¿De qué verdad habla?

Tu ciencia ficción esta muy fumada, chance deberíamos meternos lo que tu te metes para e

In [ ]:
#username = WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'input[autocomplete="username"]')))
#username.send_keys("your_username")
#username.send_keys(Keys.ENTER)


In [ ]:
'''
import time
from selenium import webdriver
from selenium.webdriver import ChromeOptions, Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait


options = ChromeOptions()
options.add_argument("--start-maximized")
options.add_experimental_option("excludeSwitches", ["enable-automation"])

driver = webdriver.Chrome(options=options)
url = "https://twitter.com/i/flow/login"
driver.get(url)

username = WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'input[autocomplete="username"]')))
username.send_keys("your_username")
username.send_keys(Keys.ENTER)

password = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'input[name="password"]')))
password.send_keys("your_password")
password.send_keys(Keys.ENTER)

time.sleep(10)
'''

In [ ]:
x = 'https://x.com/i/flow/login'